# Mobile App Funnel — 3-Month Analysis Notebook

This notebook loads the **eight event CSVs**, performs schema normalization, and produces:

1. Data audit per file (rows, unique users, timestamp ranges, duplicates, nulls)
2. Canonical event log (`events_clean`)
3. A month-by-month **funnel** (unique users) for the **last 3 complete calendar months**
4. Step-to-step conversion % and leak points
5. **Service-type splits** *(Walk vs Sitting vs Other)* **for reliable months only** (excludes months where `BeforePurchase` was tracked only for Walk)
6. Outputs to `outputs/` folder: CSVs + an Excel report

**Assumptions confirmed by stakeholders:**
- Walk → {AdHoc, Planned, Package, Customize}
- Sitting → {Boarding, Sitting, CatBoarding, CatSitting}
- `BeforePurchase` serviceType was Walk-only until ~1 month ago; months dominated by `Walking` are excluded from service splits
- *Pet added* step counts users having **dog OR cat** added
- We count **unique users** per step per month (purchase totals per user can be >1; we also provide raw order counts separately)

Run cells top-to-bottom. Set `DATA_DIR` below if your CSVs are elsewhere.


In [1]:
# Configuration
from pathlib import Path
import pandas as pd
import numpy as np
import pytz
from datetime import datetime

# ---- Change this if your CSVs are in another folder ----
DATA_DIR = Path('/Users/tree/Projects/tubitak-ai-agent/tubitakaiagentprojeleriiinverisetleri')  # e.g., Path('/path/to/csvs')

IST_TZ = pytz.timezone('Europe/Istanbul')

# Expected files and their primary columns
CONFIG = {
    'SignupCompleted.csv':       {'time_col': 'signuptime',          'id_cols': ['id','user_id','ownerid'], 'event': 'signup_completed'},
    'AddressAdded.csv':          {'time_col': 'addressaddedtime',    'id_cols': ['ownerid','user_id','id'], 'event': 'address_added'},
    'DogAdded.csv':              {'time_col': 'dogaddedtime',        'id_cols': ['ownerid','user_id','id'], 'event': 'dog_added'},
    'CatAdded.csv':              {'time_col': 'cataddedtime',        'id_cols': ['ownerid','user_id','id'], 'event': 'cat_added'},
    'CreditcardAdded.csv':       {'time_col': 'cardaddedtime',       'id_cols': ['ownerid','user_id','id'], 'event': 'card_added'},
    'CheckoutPageOpened.csv':    {'time_col': 'event_time',          'id_cols': ['user_id','ownerid','id'], 'event': 'checkout_opened'},
    'BeforePurchaseDetailsScreen.csv':{'time_col': 'event_time',     'id_cols': ['user_id','ownerid','id'], 'event': 'before_purchase'},
    'Purchase.csv':              {'time_col': 'ordercreatedtime',    'id_cols': ['ownerid','user_id','id'], 'event': 'purchase'}
}

# Canonical funnel order
FUNNEL_STEPS = [
    'signup_completed', 'address_added', 'pet_added', 'card_added',
    'checkout_opened', 'before_purchase', 'purchase'
]

# Service taxonomy (confirmed)
WALK_TYPES = {'AdHoc','Planned','Package','Customize'}
SITTING_TYPES = {'Boarding','Sitting','CatBoarding','CatSitting'}

OUTPUT_DIR = Path('outputs')
OUTPUT_DIR.mkdir(exist_ok=True)

In [2]:
# Helper functions
def _coalesce_id(df: pd.DataFrame, candidates):
    for c in candidates:
        if c in df.columns:
            return df[c]
    return pd.Series([np.nan]*len(df), index=df.index)

def _parse_time_series(s: pd.Series, assume_local_tz=IST_TZ):
    # Try parse without forcing UTC
    t = pd.to_datetime(s, errors='coerce', infer_datetime_format=True)
    # If tz-naive, localize to Istanbul then convert to UTC
    if getattr(t.dt, 'tz', None) is None:
        try:
            t = t.dt.tz_localize(assume_local_tz, nonexistent='NaT', ambiguous='NaT').dt.tz_convert('UTC')
        except Exception:
            # Fallback: treat as UTC if localization fails
            t = pd.to_datetime(s, errors='coerce', utc=True)
    else:
        t = t.dt.tz_convert('UTC')
    return t

def load_events(data_dir: Path):
    frames = {}
    issues = []
    for fname, cfg in CONFIG.items():
        fpath = data_dir / fname
        if not fpath.exists():
            issues.append(f"Missing file: {fname}")
            continue
        try:
            df = pd.read_csv(fpath)
            df['user_id'] = _coalesce_id(df, cfg['id_cols']).astype(str)
            # normalize serviceType spelling/case if present
            st_col = None
            for c in ['serviceType','servicetype','service_type']:
                if c in df.columns:
                    st_col = c
                    break
            df['serviceType'] = df[st_col].astype(str) if st_col else np.nan
            # parse time
            if cfg['time_col'] not in df.columns:
                issues.append(f"{fname}: time column '{cfg['time_col']}' not found")
                continue
            t = _parse_time_series(df[cfg['time_col']])
            df['event_time'] = t
            df['event_name'] = cfg['event']
            frames[fname] = df
        except Exception as e:
            issues.append(f"Error loading {fname}: {e}")
    return frames, issues

def audit_tables(frames):
    rows = []
    for fname, df in frames.items():
        n_dups = int(df.duplicated(subset=['user_id','event_time','event_name']).sum())
        rows.append({
            'file': fname,
            'event': df['event_name'].iloc[0],
            'rows': len(df),
            'unique_users': df['user_id'].nunique(),
            'duplicate_rows': n_dups,
            'null_user_ids': int(df['user_id'].isna().sum() or (df['user_id']=='nan').sum()),
            'null_timestamps': int(df['event_time'].isna().sum()),
            'time_min_istanbul': str(df['event_time'].min().tz_convert(IST_TZ)) if not df['event_time'].isna().all() else None,
            'time_max_istanbul': str(df['event_time'].max().tz_convert(IST_TZ)) if not df['event_time'].isna().all() else None,
            'serviceType_non_null': int(df['serviceType'].notna().sum()),
            'serviceType_distinct': int(df['serviceType'].nunique(dropna=True))
        })
    audit_df = pd.DataFrame(rows).sort_values('file')
    st_rows = []
    for fname, df in frames.items():
        vc = df['serviceType'].value_counts(dropna=True)
        for st, cnt in vc.items():
            st_rows.append({'file': fname, 'serviceType': st, 'count': int(cnt)})
    st_df = pd.DataFrame(st_rows).sort_values(['file','count'], ascending=[True,False])
    return audit_df, st_df

def build_canonical_events(frames):
    ev = pd.concat(frames.values(), ignore_index=True)
    # Deduplicate exact event duplicates
    ev = ev.drop_duplicates(subset=['user_id','event_time','event_name'])
    # Build pet_added (union dog|cat)
    pets = ev[ev['event_name'].isin(['dog_added','cat_added'])].copy()
    pets['event_name'] = 'pet_added'
    pets = pets.drop_duplicates(subset=['user_id','event_time','event_name'])
    # Combine
    ev2 = pd.concat([ev, pets], ignore_index=True)
    # Service category mapping
    def map_cat(st):
        if pd.isna(st):
            return np.nan
        if st in WALK_TYPES:
            return 'Walk'
        if st in SITTING_TYPES:
            return 'Sitting'
        return 'Other'
    ev2['service_category'] = ev2['serviceType'].apply(map_cat)
    return ev2

def last_3_complete_months(anchor_time):
    # anchor_time is UTC tz-aware
    anchor = pd.Timestamp(year=anchor_time.year, month=anchor_time.month, day=1, tz='UTC')
    start = anchor - pd.offsets.MonthBegin(3)   # start of 3 months window
    end = anchor + pd.offsets.MonthBegin(1)     # start of the next month (exclusive)
    return start, end

def monthly_uniques(ev, steps, start, end):
    m = ev[(ev['event_time']>=start) & (ev['event_time']<end)].copy()
    m['month'] = m['event_time'].dt.tz_convert(IST_TZ).dt.to_period('M').astype(str)
    counts = (m[m['event_name'].isin(steps)]
              .groupby(['month','event_name'])['user_id'].nunique()
              .reset_index(name='unique_users'))
    pivot = counts.pivot(index='month', columns='event_name', values='unique_users').fillna(0).astype(int)
    # Ensure step columns order
    for s in steps:
        if s not in pivot.columns:
            pivot[s] = 0
    pivot = pivot[steps]
    return pivot.reset_index()

def conversion_metrics(funnel_df, steps):
    df = funnel_df.copy()
    # step-to-step conversion and cumulative from first step
    for i in range(1, len(steps)):
        prev_s, cur_s = steps[i-1], steps[i]
        df[f'{prev_s}_to_{cur_s}_rate'] = np.where(df[prev_s]>0, df[cur_s]/df[prev_s], np.nan)
    for s in steps[1:]:
        df[f'cumulative_{s}_rate'] = np.where(df[steps[0]]>0, df[s]/df[steps[0]], np.nan)
    return df

def compute_leaks(funnel_df, steps):
    # leak = 1 - step_to_step_rate
    leak_cols = {}
    for i in range(1, len(steps)):
        prev_s, cur_s = steps[i-1], steps[i]
        col = f'leak_{prev_s}_to_{cur_s}'
        leak_cols[col] = 1 - np.where(funnel_df[prev_s]>0, funnel_df[cur_s]/funnel_df[prev_s], np.nan)
    leak_df = funnel_df[['month']].copy()
    for col, series in leak_cols.items():
        leak_df[col] = series
    return leak_df

def beforepurchase_reliable_months(ev):
    bp = ev[ev['event_name']=='before_purchase'].dropna(subset=['event_time']).copy()
    if bp.empty:
        return []
    bp['month'] = bp['event_time'].dt.tz_convert(IST_TZ).dt.to_period('M').astype(str)
    distr = (bp.groupby(['month','service_category'])
               .size().reset_index(name='events'))
    month_tot = distr.groupby('month')['events'].transform('sum')
    distr['share'] = distr['events']/month_tot
    # Reliable if Walk is not overwhelmingly dominant (e.g., < 0.95 share)
    walk_share = distr[distr['service_category']=='Walk'].set_index('month')['share']
    months = sorted(distr['month'].unique())
    reliable = []
    for m in months:
        ws = walk_share.get(m, 0.0)
        if ws < 0.95:  # heuristic threshold
            reliable.append(m)
    return reliable

def service_split_funnel(ev, steps, months_subset):
    m = ev.copy()
    m['month'] = m['event_time'].dt.tz_convert(IST_TZ).dt.to_period('M').astype(str)
    if months_subset:
        m = m[m['month'].isin(months_subset)]
    # Group by service_category (including NaN/Other)
    m['service_bucket'] = m['service_category'].fillna('Unknown')
    counts = (m[m['event_name'].isin(steps)]
              .groupby(['month','service_bucket','event_name'])['user_id']
              .nunique().reset_index(name='unique_users'))
    # Pivot per month & bucket
    pivot = counts.pivot_table(index=['month','service_bucket'], columns='event_name', values='unique_users', fill_value=0)
    # Ensure all step columns exist
    for s in steps:
        if s not in pivot.columns:
            pivot[s] = 0
    pivot = pivot[steps].reset_index()
    return pivot

In [3]:
# Load & audit
frames, issues = load_events(DATA_DIR)
if issues:
    print('Issues found during load:')
    for msg in issues:
        print(' -', msg)
audit_df, st_df = audit_tables(frames)
display(audit_df)
display(st_df.head(50))

# Build canonical event log
events = build_canonical_events(frames)
events_clean = events.dropna(subset=['event_time','user_id']).copy()
events_clean.to_csv(OUTPUT_DIR / 'events_clean.csv', index=False)
print('Saved canonical events to', OUTPUT_DIR / 'events_clean.csv')
events_clean.head()

/var/folders/_y/5xqwn5xx5_s9dsm9q1ymghdr0000gn/T/ipykernel_4867/891571958.py:10: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  t = pd.to_datetime(s, errors='coerce', infer_datetime_format=True)
/var/folders/_y/5xqwn5xx5_s9dsm9q1ymghdr0000gn/T/ipykernel_4867/891571958.py:10: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  t = pd.to_datetime(s, errors='coerce', infer_datetime_format=True)
/var/folders/_y/5xqwn5xx5_s9dsm9q1ymghdr0000gn/T/ipykernel_4867/891571958.py:10: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a futu

,file,event,rows,unique_users,duplicate_rows,null_user_ids,null_timestamps,time_min_istanbul,time_max_istanbul,serviceType_non_null,serviceType_distinct
1,AddressAdded.csv,address_added,1917,1771,0,0,0,2025-05-06 13:45:53.237000+03:00,2025-08-03 23:33:52.886000+03:00,0,0
6,BeforePurchaseDetailsScreen.csv,before_purchase,36911,3902,21701,21702,21702,2025-07-02 14:42:25.971000+03:00,2025-08-04 11:59:43.172000+03:00,36911,12
3,CatAdded.csv,cat_added,463,401,0,0,0,2025-05-06 19:12:02.393000+03:00,2025-08-03 15:17:00.688000+03:00,0,0
5,CheckoutPageOpened.csv,checkout_opened,31478,3213,0,0,0,2025-05-06 13:49:55.880000+03:00,2025-08-04 11:59:07.519000+03:00,31478,12
4,CreditcardAdded.csv,card_added,725,643,0,0,0,2025-05-06 13:52:44.516000+03:00,2025-08-03 21:19:50.717000+03:00,0,0
2,DogAdded.csv,dog_added,1707,1532,0,0,0,2025-05-06 14:44:53.187000+03:00,2025-08-03 23:30:28.485000+03:00,0,0
7,Purchase.csv,purchase,28770,1646,14417,2,69,2024-12-24 21:41:02.022000+03:00,2025-08-04 09:34:52.801000+03:00,28770,12
0,SignupCompleted.csv,signup_completed,10986,10986,0,0,0,2025-05-06 13:38:39.450000+03:00,2025-08-04 00:15:42.778000+03:00,0,0


,file,serviceType,count
12,BeforePurchaseDetailsScreen.csv,nan,21702
13,BeforePurchaseDetailsScreen.csv,Planned,5386
14,BeforePurchaseDetailsScreen.csv,AdHoc,3660
15,BeforePurchaseDetailsScreen.csv,Customize,2632
16,BeforePurchaseDetailsScreen.csv,Boarding,937
17,BeforePurchaseDetailsScreen.csv,Sitting,814
18,BeforePurchaseDetailsScreen.csv,CatSitting,522
19,BeforePurchaseDetailsScreen.csv,Grooming,378
20,BeforePurchaseDetailsScreen.csv,Veterinary,289
21,BeforePurchaseDetailsScreen.csv,TrainingService,281


Saved canonical events to outputs/events_clean.csv


,id,signuptime,user_id,serviceType,event_time,event_name,addressid,ownerid,province,district,...,cardaddedtime,uuid,uuid.1,user_id.1,event_time.1,serviceType.1,serviceid,ordercreatedtime,servicetype,service_category
0,4c73df87-0261-49b4-8093-ad005d847a73,2025-06-05 12:37:38.570,4c73df87-0261-49b4-8093-ad005d847a73,NaN,2025-06-05 09:37:38.570000+00:00,signup_completed,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e3ad73cc-069e-40b8-88de-9e9b809f0f30,2025-06-10 11:28:09.529,e3ad73cc-069e-40b8-88de-9e9b809f0f30,NaN,2025-06-10 08:28:09.529000+00:00,signup_completed,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,57e12ad5-86f6-4dbe-aaaf-f0053ab0177c,2025-07-25 12:29:12.305,57e12ad5-86f6-4dbe-aaaf-f0053ab0177c,NaN,2025-07-25 09:29:12.305000+00:00,signup_completed,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17c461c9-e459-44e2-a9cf-108ed7596d48,2025-05-23 14:46:42.714,17c461c9-e459-44e2-a9cf-108ed7596d48,NaN,2025-05-23 11:46:42.714000+00:00,signup_completed,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3e0ea1ea-82f0-4a06-9505-6a0b8ba789ee,2025-07-01 01:01:24.759,3e0ea1ea-82f0-4a06-9505-6a0b8ba789ee,NaN,2025-06-30 22:01:24.759000+00:00,signup_completed,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Determine 3 complete months window based on max timestamp
if len(events_clean)==0:
    raise ValueError('No events after cleaning.')
max_time = events_clean['event_time'].max()
start, end = last_3_complete_months(max_time)
print('Window (UTC):', start, '→', end, '(exclusive)')
print('Window (Istanbul):', start.tz_convert(IST_TZ), '→', end.tz_convert(IST_TZ))

Window (UTC): 2025-05-01 00:00:00+00:00 → 2025-09-01 00:00:00+00:00 (exclusive)
Window (Istanbul): 2025-05-01 03:00:00+03:00 → 2025-09-01 03:00:00+03:00


In [5]:
# Monthly unique users per step (last 3 months)
funnel_mu = monthly_uniques(events_clean, FUNNEL_STEPS, start, end)
display(funnel_mu)

# Conversion and leak metrics
funnel_conv = conversion_metrics(funnel_mu, FUNNEL_STEPS)
leaks_df = compute_leaks(funnel_mu, FUNNEL_STEPS)
display(funnel_conv)
display(leaks_df)

# Save
funnel_mu.to_csv(OUTPUT_DIR / 'funnel_monthly_uniques.csv', index=False)
funnel_conv.to_csv(OUTPUT_DIR / 'funnel_conversions.csv', index=False)
leaks_df.to_csv(OUTPUT_DIR / 'funnel_leaks.csv', index=False)
print('Saved funnel tables to outputs/.')

/var/folders/_y/5xqwn5xx5_s9dsm9q1ymghdr0000gn/T/ipykernel_4867/891571958.py:109: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  m['month'] = m['event_time'].dt.tz_convert(IST_TZ).dt.to_period('M').astype(str)


event_name,month,signup_completed,address_added,pet_added,card_added,checkout_opened,before_purchase,purchase
0,2025-05,2492,486,528,194,1500,0,970
1,2025-06,4027,573,670,233,1652,0,975
2,2025-07,4184,676,634,220,1543,3595,864
3,2025-08,283,60,69,28,373,603,232


event_name,month,signup_completed,address_added,pet_added,card_added,checkout_opened,before_purchase,purchase,signup_completed_to_address_added_rate,address_added_to_pet_added_rate,pet_added_to_card_added_rate,card_added_to_checkout_opened_rate,checkout_opened_to_before_purchase_rate,before_purchase_to_purchase_rate,cumulative_address_added_rate,cumulative_pet_added_rate,cumulative_card_added_rate,cumulative_checkout_opened_rate,cumulative_before_purchase_rate,cumulative_purchase_rate
0,2025-05,2492,486,528,194,1500,0,970,0.195024,1.086420,0.367424,7.731959,0.000000,NaN,0.195024,0.211878,0.077849,0.601926,0.000000,0.389246
1,2025-06,4027,573,670,233,1652,0,975,0.142290,1.169284,0.347761,7.090129,0.000000,NaN,0.142290,0.166377,0.057859,0.410231,0.000000,0.242116
2,2025-07,4184,676,634,220,1543,3595,864,0.161568,0.937870,0.347003,7.013636,2.329877,0.240334,0.161568,0.151530,0.052581,0.368786,0.859226,0.206501
3,2025-08,283,60,69,28,373,603,232,0.212014,1.150000,0.405797,13.321429,1.616622,0.384743,0.212014,0.243816,0.098940,1.318021,2.130742,0.819788


event_name,month,leak_signup_completed_to_address_added,leak_address_added_to_pet_added,leak_pet_added_to_card_added,leak_card_added_to_checkout_opened,leak_checkout_opened_to_before_purchase,leak_before_purchase_to_purchase
0,2025-05,0.804976,-0.086420,0.632576,-6.731959,1.000000,NaN
1,2025-06,0.857710,-0.169284,0.652239,-6.090129,1.000000,NaN
2,2025-07,0.838432,0.062130,0.652997,-6.013636,-1.329877,0.759666
3,2025-08,0.787986,-0.150000,0.594203,-12.321429,-0.616622,0.615257


Saved funnel tables to outputs/.


In [6]:
# Service-type reliability & split funnels
reliable_months = beforepurchase_reliable_months(events_clean)
print('Reliable months for service split at BeforePurchase (heuristic):', reliable_months)

split_df = service_split_funnel(events_clean, FUNNEL_STEPS, months_subset=reliable_months)
display(split_df.head(30))
split_df.to_csv(OUTPUT_DIR / 'funnel_by_service_bucket.csv', index=False)
print('Saved service-split funnel to outputs/.')

Reliable months for service split at BeforePurchase (heuristic): ['2025-07', '2025-08']


/var/folders/_y/5xqwn5xx5_s9dsm9q1ymghdr0000gn/T/ipykernel_4867/891571958.py:147: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  bp['month'] = bp['event_time'].dt.tz_convert(IST_TZ).dt.to_period('M').astype(str)
/var/folders/_y/5xqwn5xx5_s9dsm9q1ymghdr0000gn/T/ipykernel_4867/891571958.py:164: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  m['month'] = m['event_time'].dt.tz_convert(IST_TZ).dt.to_period('M').astype(str)


event_name,month,service_bucket,signup_completed,address_added,pet_added,card_added,checkout_opened,before_purchase,purchase
0,2025-07,Other,0.0,0.0,0.0,0.0,190.0,581.0,46.0
1,2025-07,Sitting,0.0,0.0,0.0,0.0,520.0,1267.0,110.0
2,2025-07,Unknown,4184.0,676.0,634.0,220.0,0.0,0.0,0.0
3,2025-07,Walk,0.0,0.0,0.0,0.0,1100.0,2692.0,773.0
4,2025-08,Other,0.0,0.0,0.0,0.0,38.0,87.0,6.0
5,2025-08,Sitting,0.0,0.0,0.0,0.0,71.0,172.0,11.0
6,2025-08,Unknown,283.0,60.0,69.0,28.0,0.0,0.0,0.0
7,2025-08,Walk,0.0,0.0,0.0,0.0,294.0,451.0,217.0


Saved service-split funnel to outputs/.


In [7]:
# (Optional) raw order counts per month, in addition to unique purchasing users
orders = events_clean[events_clean['event_name']=='purchase'].copy()
orders['month'] = orders['event_time'].dt.tz_convert(IST_TZ).dt.to_period('M').astype(str)
orders_users = orders.groupby('month')['user_id'].nunique().reset_index(name='purchasing_users')
orders_cnt = orders.groupby('month').size().reset_index(name='order_events')
orders_summary = orders_users.merge(orders_cnt, on='month', how='outer').sort_values('month')
display(orders_summary.tail(6))
orders_summary.to_csv(OUTPUT_DIR / 'orders_summary.csv', index=False)
print('Saved orders_summary.csv')

/var/folders/_y/5xqwn5xx5_s9dsm9q1ymghdr0000gn/T/ipykernel_4867/3179813195.py:3: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  orders['month'] = orders['event_time'].dt.tz_convert(IST_TZ).dt.to_period('M').astype(str)


,month,purchasing_users,order_events
3,2025-03,19,23
4,2025-04,105,128
5,2025-05,970,4775
6,2025-06,975,4727
7,2025-07,864,4220
8,2025-08,232,396


Saved orders_summary.csv


In [8]:
# Write an Excel report aggregating the key tables (robust engine + tz-naive datetimes)
import pandas as pd

def excel_safe(df, to_tz="Europe/Istanbul"):
    """Return a copy with all tz-aware datetime columns converted to naive in the given TZ."""
    out = df.copy()
    for col in out.columns:
        if pd.api.types.is_datetime64tz_dtype(out[col]):
            out[col] = out[col].dt.tz_convert(to_tz).dt.tz_localize(None)
    return out

# pick engine: prefer xlsxwriter; fall back to openpyxl
try:
    import xlsxwriter  # noqa: F401
    EXCEL_ENGINE = "xlsxwriter"
except ImportError:
    try:
        import openpyxl  # noqa: F401
        EXCEL_ENGINE = "openpyxl"
    except ImportError:
        raise RuntimeError(
            "No Excel writer installed. Install one of:\n"
            "  pip install XlsxWriter   # fast, charts supported\n"
            "  pip install openpyxl     # pure Python"
        )

excel_path = OUTPUT_DIR / 'funnel_report.xlsx'
with pd.ExcelWriter(excel_path, engine=EXCEL_ENGINE) as writer:
    excel_safe(audit_df).to_excel(writer, sheet_name='01_audit', index=False)
    excel_safe(st_df).to_excel(writer, sheet_name='02_serviceType_values', index=False)
    excel_safe(events_clean.head(1000)).to_excel(writer, sheet_name='03_events_sample', index=False)
    excel_safe(funnel_mu).to_excel(writer, sheet_name='04_funnel_last3m_uniques', index=False)
    excel_safe(funnel_conv).to_excel(writer, sheet_name='05_conversion_rates', index=False)
    excel_safe(leaks_df).to_excel(writer, sheet_name='06_leaks', index=False)
    excel_safe(split_df).to_excel(writer, sheet_name='07_by_service_bucket', index=False)
    excel_safe(orders_summary).to_excel(writer, sheet_name='08_orders_summary', index=False)

excel_path

/var/folders/_y/5xqwn5xx5_s9dsm9q1ymghdr0000gn/T/ipykernel_4867/1363291657.py:8: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if pd.api.types.is_datetime64tz_dtype(out[col]):


PosixPath('outputs/funnel_report.xlsx')

## Notes & Caveats
- **Monthly uniques funnel**: counts are **not** strict step-progressions. A user can appear in a later step even if their earlier step happened in another month. For strict cohort progression, we would compute first-seen timestamps per user per step and analyze by signup cohort.
- **BeforePurchase coverage**: months dominated by `Walk` are excluded from service splits. Adjust the 0.95 threshold in `beforepurchase_reliable_months` if you have a known switch date.
- **Timezones**: naive timestamps are assumed to be Istanbul local. If your raw data is UTC, set that in `_parse_time_series` accordingly.
- **Pet step**: `pet_added` is derived from union of `dog_added` and `cat_added`.
- **Multiple orders per user**: funnel uses unique users; see `orders_summary.csv` for raw counts.
